# Comparative Analysis of Air Quality Variability Between Major and Peripheral Cities: A Case Study of Thailand

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene
from itertools import product

## Import dataset

In [2]:
df = pd.read_csv("thai_air_data_cleaned.csv")

In [3]:
df.head()

,Timestamp,City,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time
0,2025-02-19 18:11:33,Samut Prakan,"City Hall, Samut Prakan, Thailand (ศาลากลางจัง...",13.599149,100.597345,0.1,-5.9,8.4,9.3,13.4,1008.5,55.0,110.0,0.6,0.6,31.7,0.5,2025-02-20 00:00:00
1,2025-02-19 18:11:33,Nakhon Ratchasima,"Municipal Waste Water Pumping Station, Nakhon ...",14.979537,102.098335,NaN,-4.6,9.2,11.0,23.7,985.9,NaN,100.0,0.2,0.6,31.9,0.5,2025-02-20 00:00:00
2,2025-02-19 18:11:34,Surin,"Pyramid House, Surin, Thailand",14.793612,103.430560,NaN,-4.6,9.2,NaN,NaN,993.9,NaN,NaN,NaN,NaN,32.0,0.5,NaN
3,2025-02-19 18:11:34,Sakon Nakhon,"Meteorological stations, Sakon Nakhon, Thailan...",17.156620,104.133216,0.1,-4.9,9.9,NaN,20.1,991.0,43.0,63.0,NaN,1.1,30.2,1.5,2025-02-20 00:00:00
4,2025-02-19 18:11:35,Mueang Khon Kaen,"Khonkaen, Thailand (ส่วนอุทกวิทยา สำนักงานทรัพ...",16.445383,102.835250,NaN,-5.2,9.0,NaN,NaN,988.0,NaN,79.0,0.6,NaN,31.5,0.5,2025-02-20 00:00:00


In [4]:
df.shape

(1866, 18)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  1866 non-null   object 
 1   City       1866 non-null   object 
 2   name       1866 non-null   object 
 3   lat        1866 non-null   float64
 4   lon        1866 non-null   float64
 5   co         1199 non-null   float64
 6   dew        1574 non-null   float64
 7   h          1866 non-null   float64
 8   no2        1439 non-null   float64
 9   o3         1484 non-null   float64
 10  p          1866 non-null   float64
 11  pm10       1594 non-null   float64
 12  pm25       1812 non-null   float64
 13  r          912 non-null    float64
 14  so2        1376 non-null   float64
 15  t          1866 non-null   float64
 16  w          1866 non-null   float64
 17  time       1812 non-null   object 
dtypes: float64(14), object(4)
memory usage: 262.5+ KB


In [6]:
df["time"] = pd.to_datetime(df["time"])

In [7]:
df.isna().sum()

Timestamp      0
City           0
name           0
lat            0
lon            0
co           667
dew          292
h              0
no2          427
o3           382
p              0
pm10         272
pm25          54
r            954
so2          490
t              0
w              0
time          54
dtype: int64

In [8]:
(df.groupby(["City"]).apply(lambda x: x.isna().sum() / len(x) * 100, include_groups=False)
                     .drop(columns="City", errors="ignore").round(2))

,Timestamp,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time
City,,,,,,,,,,,,,,,,,
Ayutthaya,0.0,0.0,0.0,0.0,0.00,15.09,0.0,0.0,0.00,0.0,0.00,0.0,60.38,0.00,0.0,0.0,0.0
Bangkok,0.0,0.0,0.0,0.0,0.00,100.00,0.0,0.0,0.00,0.0,0.00,0.0,24.53,0.00,0.0,0.0,0.0
Chiang Mai,0.0,0.0,0.0,0.0,100.00,100.00,0.0,0.0,0.00,0.0,0.00,0.0,73.58,0.00,0.0,0.0,0.0
Chonburi,0.0,0.0,0.0,0.0,100.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,36.54,0.00,0.0,0.0,0.0
Kamphaeng Phet,0.0,0.0,0.0,0.0,100.00,100.00,0.0,100.0,100.00,0.0,0.00,0.0,100.00,100.00,0.0,0.0,0.0
Kanchanaburi,0.0,0.0,0.0,0.0,0.00,0.00,0.0,100.0,0.00,0.0,0.00,0.0,40.00,100.00,0.0,0.0,0.0
Krabi,0.0,0.0,0.0,0.0,100.00,0.00,0.0,100.0,100.00,0.0,100.00,0.0,100.00,100.00,0.0,0.0,0.0
Lampang,0.0,0.0,0.0,0.0,0.00,20.75,0.0,0.0,0.00,0.0,0.00,0.0,54.72,0.00,0.0,0.0,0.0
Lamphun,0.0,0.0,0.0,0.0,100.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,79.63,0.00,0.0,0.0,0.0


In [9]:
df.groupby(["City"]).size()

City
Ayutthaya              53
Bangkok                53
Chiang Mai             53
Chonburi               52
Kamphaeng Phet          1
Kanchanaburi           55
Krabi                   1
Lampang                53
Lamphun                54
Mueang Chiang Rai      53
Mueang Khon Kaen       52
Nakhon Pathom          52
Nakhon Ratchasima      55
Nakhon Sawan           55
Nan                    54
Narathiwat             52
Nong Khai              55
Nonthaburi             54
Pathum Thani           52
Pattani                 1
Phayao                 54
Phitsanulok            54
Phuket                 52
Prachuap Khiri Khan    53
Ratchaburi             52
Rayong                 52
Sakon Nakhon           54
Samut Prakan           55
Samut Sakhon           53
Saraburi               52
Satun                  52
Suphan Buri            55
Surin                  54
Trang                  52
Trat                   53
Ubon Ratchathani       53
Uthai Thani             1
Uttaradit              53
Yala   

In [10]:
# remove cities with only one record
df = df[~df["City"].isin(["Kamphaeng Phet", "Krabi", "Pattani", "Uthai Thani", "Surin"])]

df.groupby(["City"]).size()

City
Ayutthaya              53
Bangkok                53
Chiang Mai             53
Chonburi               52
Kanchanaburi           55
Lampang                53
Lamphun                54
Mueang Chiang Rai      53
Mueang Khon Kaen       52
Nakhon Pathom          52
Nakhon Ratchasima      55
Nakhon Sawan           55
Nan                    54
Narathiwat             52
Nong Khai              55
Nonthaburi             54
Pathum Thani           52
Phayao                 54
Phitsanulok            54
Phuket                 52
Prachuap Khiri Khan    53
Ratchaburi             52
Rayong                 52
Sakon Nakhon           54
Samut Prakan           55
Samut Sakhon           53
Saraburi               52
Satun                  52
Suphan Buri            55
Trang                  52
Trat                   53
Ubon Ratchathani       53
Uttaradit              53
Yala                   52
dtype: int64

## Data preprocessing

### Define city type (major/peripheral) and region (North, Northeast, East, Central, West, South)

In [11]:
# City type: major and peripheral
city_type = {
    'Samut Prakan': 'Peripheral',
    'Nakhon Ratchasima': 'Major',
    'Surin': 'Peripheral',
    'Sakon Nakhon': 'Peripheral',
    'Mueang Khon Kaen': 'Major',
    'Nong Khai': 'Peripheral',
    'Kanchanaburi': 'Major',
    'Suphan Buri': 'Peripheral',
    'Uthai Thani': 'Peripheral',
    'Nakhon Sawan': 'Peripheral',
    'Phitsanulok': 'Peripheral',
    'Kamphaeng Phet': 'Peripheral',
    'Lamphun': 'Peripheral',
    'Lampang': 'Peripheral',
    'Phayao': 'Peripheral',
    'Bangkok': 'Major',
    'Trat': 'Peripheral',
    'Nonthaburi': 'Peripheral',
    'Nan': 'Peripheral',
    'Ubon Ratchathani': 'Peripheral',
    'Samut Sakhon': 'Peripheral',
    'Chiang Mai': 'Major',
    'Mueang Chiang Rai': 'Peripheral',
    'Ayutthaya': 'Peripheral',
    'Uttaradit': 'Peripheral',
    'Rayong': 'Major',
    'Pathum Thani': 'Peripheral',
    'Chonburi': 'Major',
    'Nakhon Pathom': 'Peripheral',
    'Ratchaburi': 'Peripheral',
    'Phuket': 'Major',
    'Yala': 'Peripheral',
    'Krabi': 'Peripheral',
    'Trang': 'Peripheral',
    'Satun': 'Peripheral',
    'Pattani': 'Peripheral',
    'Narathiwat': 'Peripheral',
    'Prachuap Khiri Khan': 'Peripheral',
    'Saraburi': 'Peripheral',
}

df['City_type'] = df['City'].map(city_type)

In [12]:
# Region: north, northeast, central, south, west, east
city_region = {
    'Samut Prakan': 'Central',
    'Nakhon Ratchasima': 'Northeast',
    'Surin': 'Northeast',
    'Sakon Nakhon': 'Northeast',
    'Mueang Khon Kaen': 'Northeast',
    'Nong Khai': 'Northeast',
    'Kanchanaburi': 'West',
    'Suphan Buri': 'Central',
    'Uthai Thani': 'Central',
    'Nakhon Sawan': 'Central',
    'Phitsanulok': 'Central',
    'Kamphaeng Phet': 'Central',
    'Lamphun': 'North',
    'Lampang': 'North',
    'Phayao': 'North',
    'Bangkok': 'Central',
    'Trat': 'East',
    'Nonthaburi': 'Central',
    'Nan': 'North',
    'Ubon Ratchathani': 'Northeast',
    'Samut Sakhon': 'Central',
    'Chiang Mai': 'North',
    'Mueang Chiang Rai': 'North',
    'Ayutthaya': 'Central',
    'Uttaradit': 'North',
    'Rayong': 'East',
    'Pathum Thani': 'Central',
    'Chonburi': 'East',
    'Nakhon Pathom': 'Central',
    'Ratchaburi': 'West',
    'Phuket': 'South',
    'Yala': 'South',
    'Krabi': 'South',
    'Trang': 'South',
    'Satun': 'South',
    'Pattani': 'South',
    'Narathiwat': 'South',
    'Prachuap Khiri Khan': 'West',
    'Saraburi': 'Central',
}

df['City_region'] = df['City'].map(city_region)

In [13]:
df.groupby(["City_region", "City_type"])["City"].nunique()

City_region  City_type 
Central      Major          1
             Peripheral    10
East         Major          2
             Peripheral     1
North        Major          1
             Peripheral     6
Northeast    Major          2
             Peripheral     3
South        Major          1
             Peripheral     4
West         Major          1
             Peripheral     2
Name: City, dtype: int64

### Impute missing value

In [14]:
df_no_missing = df.sort_values(by=["City", "time"]).copy()
df_no_missing = df_no_missing.groupby(["City"]).apply(lambda g: g.ffill().bfill(), include_groups=False)
df_no_missing = df_no_missing.reset_index(drop=False)
df_no_missing = df_no_missing.drop(columns=["level_1"])
df_no_missing

,City,Timestamp,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time,City_type,City_region
0,Ayutthaya,2025-02-19 18:11:46,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,-5.9,9.5,3.5,5.2,1008.4,64.0,74.0,0.5,0.6,29.6,2.5,2025-02-20 00:00:00,Peripheral,Central
1,Ayutthaya,2025-02-20 12:00:25,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,23.8,69.5,3.5,5.2,1013.9,64.0,135.0,0.5,0.6,30.0,2.5,2025-02-20 18:00:00,Peripheral,Central
2,Ayutthaya,2025-02-21 01:36:00,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,84.8,3.5,5.2,1011.4,64.0,138.0,0.5,0.6,25.8,1.0,2025-02-21 08:00:00,Peripheral,Central
3,Ayutthaya,2025-02-22 01:53:38,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,76.0,3.5,5.2,1012.9,64.0,95.0,0.5,0.6,27.2,0.5,2025-02-22 08:00:00,Peripheral,Central
4,Ayutthaya,2025-02-22 02:11:00,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,76.0,3.5,5.2,1012.4,64.0,88.0,0.5,0.6,26.6,0.2,2025-02-22 09:00:00,Peripheral,Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,Yala,2025-04-07 14:01:14,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,-6.6,8.0,NaN,NaN,1006.7,33.0,40.0,0.2,NaN,31.8,1.5,2025-04-07 19:00:00,Peripheral,South
1804,Yala,2025-04-08 15:09:19,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,24.4,66.5,NaN,NaN,1008.0,33.0,46.0,0.2,NaN,31.4,3.6,2025-04-08 21:00:00,Peripheral,South
1805,Yala,2025-04-09 13:00:23,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,-6.7,7.9,NaN,NaN,1009.5,33.0,42.0,0.4,NaN,31.8,3.0,2025-04-09 19:00:00,Peripheral,South
1806,Yala,2025-04-10 13:00:19,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,-6.7,7.5,NaN,NaN,1005.8,33.0,39.0,0.2,NaN,32.6,3.6,2025-04-10 19:00:00,Peripheral,South


### Resample data to day level

In [15]:
["co", "dew", "h", "no2", "o3", "p", "pm10", "pm25", "r", "so2", "t", "w"]

['co', 'dew', 'h', 'no2', 'o3', 'p', 'pm10', 'pm25', 'r', 'so2', 't', 'w']

In [16]:
df_no_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1808 entries, 0 to 1807
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   City         1808 non-null   object        
 1   Timestamp    1808 non-null   object        
 2   name         1808 non-null   object        
 3   lat          1808 non-null   float64       
 4   lon          1808 non-null   float64       
 5   co           1227 non-null   float64       
 6   dew          1597 non-null   float64       
 7   h            1808 non-null   float64       
 8   no2          1438 non-null   float64       
 9   o3           1547 non-null   float64       
 10  p            1808 non-null   float64       
 11  pm10         1753 non-null   float64       
 12  pm25         1808 non-null   float64       
 13  r            1648 non-null   float64       
 14  so2          1437 non-null   float64       
 15  t            1808 non-null   float64       
 16  w     

In [17]:
aqi_pivot = pd.pivot_table(data=df_no_missing, index="time", columns=["City"], values=["pm25", "pm10", "co", "dew", "h", "no2", "o3", "so2", "p", "r", "t", "w"], aggfunc="mean", fill_value=np.nan)
aqi_pivot

co                                                 \
City                Ayutthaya Bangkok Kanchanaburi Lampang Mueang Chiang Rai   
time                                                                           
2025-02-19 12:00:00       NaN     NaN          NaN     NaN               NaN   
2025-02-20 00:00:00       0.1     0.1          0.1     0.1               0.1   
2025-02-20 01:00:00       NaN     NaN          NaN     NaN               NaN   
2025-02-20 18:00:00       0.1     0.1          0.1     0.1               0.1   
2025-02-21 04:00:00       NaN     NaN          NaN     NaN               NaN   
...                       ...     ...          ...     ...               ...   
2025-04-09 19:00:00       0.1     0.1          0.1     0.1               0.1   
2025-04-10 06:00:00       NaN     NaN          NaN     NaN               NaN   
2025-04-10 18:00:00       NaN     NaN          NaN     NaN               NaN   
2025-04-10 19:00:00       0.1     0.1          0.1     0.1               0.1   
2025-04-11 19:00:00       0.1     0.1          0.1     0.1               0.1   

                                                                         ...  \
City                Nakhon Sawan  Nan Nong Khai Nonthaburi Pathum Thani  ...   
time                                                                     ...   
2025-02-19 12:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-02-20 00:00:00          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-20 01:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-02-20 18:00:00          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-21 04:00:00          NaN  NaN       NaN        NaN          NaN  ...   
...                          ...  ...       ...        ...          ...  ...   
2025-04-09 19:00:00          0.1  0.1       0.1        0.1          0.1  ...   
2025-04-10 06:00:00          0.1  NaN       NaN        NaN          NaN  ...   
2025-04-10 18:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-04-10 19:00:00          NaN  0.1       0.1        0.1          0.1  ...   
2025-04-11 19:00:00          0.1  0.1       0.1        0.1          0.1  ...   

                               w                                          \
City                Samut Prakan Samut Sakhon Saraburi Satun Suphan Buri   
time                                                                       
2025-02-19 12:00:00          NaN          NaN      NaN   NaN         NaN   
2025-02-20 00:00:00          0.5          2.0      0.5   2.0         2.5   
2025-02-20 01:00:00          NaN          NaN      NaN   NaN         NaN   
2025-02-20 18:00:00          1.5          1.0      0.2   3.0         2.0   
2025-02-21 04:00:00          NaN          NaN      NaN   0.5         NaN   
...                          ...          ...      ...   ...         ...   
2025-04-09 19:00:00          5.1          5.1      0.2   2.5         0.2   
2025-04-10 06:00:00          NaN          NaN      NaN   NaN         NaN   
2025-04-10 18:00:00          NaN          NaN      NaN   NaN         NaN   
2025-04-10 19:00:00          1.0          1.5      1.0   0.5         1.0   
2025-04-11 19:00:00          2.0          0.5      1.0   0.5         2.0   

                                                                
City                Trang Trat Ubon Ratchathani Uttaradit Yala  
time                                                            
2025-02-19 12:00:00   NaN  NaN              NaN       1.0  NaN  
2025-02-20 00:00:00   5.1  2.5              1.5       NaN  3.0  
2025-02-20 01:00:00   NaN  NaN              NaN       NaN  NaN  
2025-02-20 18:00:00   1.5  1.0              2.0       3.0  4.6  
2025-02-21 04:00:00   NaN  NaN              NaN       NaN  1.5  
...                   ...  ...              ...       ...  ...  
2025-04-09 19:00:00   4.1  4.1              2.0       3.6  3.0  
2025-04-10 06:00:00   NaN  NaN              NaN       NaN  NaN  
2025-

In [18]:
resampled_aqi = aqi_pivot.resample("D").mean()
resampled_aqi

co                                                 \
City       Ayutthaya Bangkok Kanchanaburi Lampang Mueang Chiang Rai   
time                                                                  
2025-02-19       NaN     NaN          NaN     NaN               NaN   
2025-02-20       0.1     0.1          0.1     0.1               0.1   
2025-02-21       0.1     0.1          0.1     0.1               0.1   
2025-02-22       0.1     0.1          0.1     0.1               0.1   
2025-02-23       0.1     0.1          0.1     0.1               0.1   
2025-02-24       0.1     0.1          0.1     0.1               0.1   
2025-02-25       0.1     0.1          0.1     0.1               0.1   
2025-02-26       0.1     0.1          0.1     0.1               0.1   
2025-02-27       0.1     0.1          0.1     0.1               0.1   
2025-02-28       0.1     0.1          0.1     0.1               0.1   
2025-03-01       0.1     0.1          0.1     0.1               0.1   
2025-03-02       0.1     0.1          0.1     0.1               0.1   
2025-03-03       0.1     0.1          0.1     0.1               0.1   
2025-03-04       0.1     0.1          0.1     0.1               0.1   
2025-03-05       0.1     0.1          0.1     0.1               0.1   
2025-03-06       0.1     0.1          0.1     0.1               0.1   
2025-03-07       0.1     0.1          0.1     0.1               0.1   
2025-03-08       0.1     0.1          0.1     0.1               0.1   
2025-03-09       0.1     0.1          0.1     0.1               0.1   
2025-03-10       0.1     0.1          0.1     0.1               0.1   
2025-03-11       0.1     0.1          0.1     0.1               0.1   
2025-03-12       0.1     0.1          NaN     0.1               0.1   
2025-03-13       0.1     0.1          0.1     0.1               0.1   
2025-03-14       NaN     NaN          NaN     NaN               NaN   
2025-03-15       NaN     NaN          NaN     NaN               NaN   
2025-03-16       NaN     NaN          0.1     0.1               NaN   
2025-03-17       NaN     NaN          NaN     NaN               NaN   
2025-03-18       0.1     0.1          0.1     0.1               0.1   
2025-03-19       0.1     0.1          0.1     0.1               0.1   
2025-03-20       0.1     0.1          0.1     0.1               0.1   
2025-03-21       0.1     0.1          0.1     0.1               0.1   
2025-03-22       NaN     NaN          NaN     NaN               NaN   
2025-03-23       0.1     0.1          0.1     0.1               0.1   
2025-03-24       0.1     0.1          0.1     0.1               0.1   
2025-03-25       0.1     0.1          0.1     0.1               0.1   
2025-03-26       0.1     0.1          0.1     0.1               0.1   
2025-03-27       NaN     0.1          0.1     0.1               0.1   
2025-03-28       0.1     0.1          0.1     0.1               0.1   
2025-03-29       0.1     0.1          0.1     NaN               0.1   
2025-03-30       NaN     NaN          NaN     0.1               NaN   
2025-03-31       0.1     0.1          0.1     0.1               0.1   
2025-04-01       0.1     0.1          0.1     0.1               0.1   
2025-04-02       0.1     0.1          0.1     0.1               0.1   
2025-04-03       0.1     0.1          0.1     0.1               0.1   
2025-04-04       0.1     0.1          0.1     0.1               0.1   
2025-04-05       0.1     0.1          0.1     0.1               0.1   
2025-04-06       NaN     NaN          NaN     NaN               NaN   
2025-04-07       0.1     0.1          0.1     0.1               0.1   
2025-04-08       0.1     0.1          0.1     0.1               0.1   
2025-04-09       0.1     0.1          0.1     0.1               0.1   
2025-04-10       0.1     0.1          0.1     0.1               0.1   
2025-04-11       0.1     0.1          0.1     0.1               0.1   

                                                                ...  \
City       Nakhon Sawan  Nan Nong Khai No

In [19]:
resampled_aqi.columns = resampled_aqi.columns.swaplevel(0, 1)
resampled_aqi = resampled_aqi.sort_index(axis=1, level=0)
resampled_aqi

City       Ayutthaya                                                     \
                  co        dew          h  no2   o3            p  pm10   
time                                                                      
2025-02-19       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-02-20       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-21       0.1  25.300000  84.800000  3.5  5.2  1011.400000  64.0   
2025-02-22       0.1  15.066667  53.633333  3.5  5.2  1013.266667  64.0   
2025-02-23       0.1   9.600000  38.700000  3.5  5.2  1013.150000  64.0   
2025-02-24       0.1  -6.100000  10.000000  3.5  5.2  1016.900000  64.0   
2025-02-25       0.1  17.800000  51.500000  3.5  5.2  1016.400000  64.0   
2025-02-26       0.1   5.850000  33.150000  3.5  5.2  1016.750000  64.0   
2025-02-27       0.1  19.000000  41.000000  3.5  5.2  1014.000000  64.0   
2025-02-28       0.1  -6.100000   7.900000  3.5  5.2  1014.000000  64.0   
2025-03-01       0.1  -6.750000   8.450000  3.5  5.2  1010.800000  64.0   
2025-03-02       0.1  -6.100000   7.600000  3.5  5.2  1009.200000  64.0   
2025-03-03       0.1  -6.500000   7.400000  3.5  5.2  1009.400000  64.0   
2025-03-04       0.1  -6.100000   7.900000  3.5  5.2  1008.800000  64.0   
2025-03-05       0.1  -6.800000   7.700000  3.5  5.2  1009.200000  64.0   
2025-03-06       0.1  -6.400000   8.000000  3.5  5.2  1010.800000  64.0   
2025-03-07       0.1  -7.300000   8.300000  3.5  5.2  1009.900000  64.0   
2025-03-08       0.1  -7.400000  11.400000  3.5  5.2  1013.800000  64.0   
2025-03-09       0.1  -7.300000  11.500000  3.5  5.2  1013.300000  64.0   
2025-03-10       0.1  -5.000000   8.200000  3.5  5.2  1014.400000  64.0   
2025-03-11       0.1  -4.900000   7.800000  3.5  5.2  1012.800000  64.0   
2025-03-12       0.1  23.000000  45.700000  3.5  5.2  1005.000000  64.0   
2025-03-13       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-14       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-15       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-16       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-17       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-18       0.1   9.500000  48.200000  3.5  5.2  1009.450000  64.0   
2025-03-19       0.1  -4.600000   9.000000  3.5  5.2  1013.800000  64.0   
2025-03-20       0.1  14.400000  39.500000  3.5  5.2  1016.100000  64.0   
2025-03-21       0.1  14.400000  49.000000  3.5  5.2  1017.100000  64.0   
2025-03-22       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-23       0.1  -4.000000  26.050000  3.5  5.2  1011.850000  64.0   
2025-03-24       0.1  21.600000  43.600000  3.5  5.2  1008.900000  64.0   
2025-03-25       0.1  -6.800000   7.700000  3.5  5.2  1008.300000  64.0   
2025-03-26       0.1  -6.900000   7.600000  3.5  5.2  1007.800000  64.0   
2025-03-27       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-28       0.1  -6.900000  48.000000  3.5  5.2  1005.500000  64.0   
2025-03-29       0.1  -5.700000   7.500000  3.5  5.2  1008.300000  64.0   
2025-03-30       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-03-31       0.1  25.450000  86.250000  3.5  5.2  1010.150000  64.0   
2025-04-01       0.1  25.700000  56.000000  3.5  5.2  1011.000000  64.0   
2025-04-02       0.1  20.400000  40.300000  3.5  5.2  1010.300000  64.0   
2025-04-03       0.1  20.400000  64.000000  3.5  5.2  1012.600000  64.0   
2025-04-04       0.1  -5.800000   7.800000  3.5  5.2  1012.600000  64.0   
2025-04-05       0.1  -5.800000   7.700000  3.5  5.2  1011.500000  64.0   
2025-04-06       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-04-07       0.1  -6.500000   8.000000  3.5  5.2  1009.350000  64.0   
2025-04-08       0.1  24.700000  55.000000  3.5  5.2  1010.700000  64.0   
2025-04-09       0.1  24.700000  64.000000  3.5  5.2  1009.800000  64.0   
2025-04-10       0.1  -6.

In [20]:
resampled_aqi = resampled_aqi.ffill().bfill()
resampled_aqi

City       Ayutthaya                                                     \
                  co        dew          h  no2   o3            p  pm10   
time                                                                      
2025-02-19       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-20       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-21       0.1  25.300000  84.800000  3.5  5.2  1011.400000  64.0   
2025-02-22       0.1  15.066667  53.633333  3.5  5.2  1013.266667  64.0   
2025-02-23       0.1   9.600000  38.700000  3.5  5.2  1013.150000  64.0   
2025-02-24       0.1  -6.100000  10.000000  3.5  5.2  1016.900000  64.0   
2025-02-25       0.1  17.800000  51.500000  3.5  5.2  1016.400000  64.0   
2025-02-26       0.1   5.850000  33.150000  3.5  5.2  1016.750000  64.0   
2025-02-27       0.1  19.000000  41.000000  3.5  5.2  1014.000000  64.0   
2025-02-28       0.1  -6.100000   7.900000  3.5  5.2  1014.000000  64.0   
2025-03-01       0.1  -6.750000   8.450000  3.5  5.2  1010.800000  64.0   
2025-03-02       0.1  -6.100000   7.600000  3.5  5.2  1009.200000  64.0   
2025-03-03       0.1  -6.500000   7.400000  3.5  5.2  1009.400000  64.0   
2025-03-04       0.1  -6.100000   7.900000  3.5  5.2  1008.800000  64.0   
2025-03-05       0.1  -6.800000   7.700000  3.5  5.2  1009.200000  64.0   
2025-03-06       0.1  -6.400000   8.000000  3.5  5.2  1010.800000  64.0   
2025-03-07       0.1  -7.300000   8.300000  3.5  5.2  1009.900000  64.0   
2025-03-08       0.1  -7.400000  11.400000  3.5  5.2  1013.800000  64.0   
2025-03-09       0.1  -7.300000  11.500000  3.5  5.2  1013.300000  64.0   
2025-03-10       0.1  -5.000000   8.200000  3.5  5.2  1014.400000  64.0   
2025-03-11       0.1  -4.900000   7.800000  3.5  5.2  1012.800000  64.0   
2025-03-12       0.1  23.000000  45.700000  3.5  5.2  1005.000000  64.0   
2025-03-13       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-14       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-15       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-16       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-17       0.1  -5.900000   7.700000  3.5  5.2  1011.200000  64.0   
2025-03-18       0.1   9.500000  48.200000  3.5  5.2  1009.450000  64.0   
2025-03-19       0.1  -4.600000   9.000000  3.5  5.2  1013.800000  64.0   
2025-03-20       0.1  14.400000  39.500000  3.5  5.2  1016.100000  64.0   
2025-03-21       0.1  14.400000  49.000000  3.5  5.2  1017.100000  64.0   
2025-03-22       0.1  14.400000  49.000000  3.5  5.2  1017.100000  64.0   
2025-03-23       0.1  -4.000000  26.050000  3.5  5.2  1011.850000  64.0   
2025-03-24       0.1  21.600000  43.600000  3.5  5.2  1008.900000  64.0   
2025-03-25       0.1  -6.800000   7.700000  3.5  5.2  1008.300000  64.0   
2025-03-26       0.1  -6.900000   7.600000  3.5  5.2  1007.800000  64.0   
2025-03-27       0.1  -6.900000   7.600000  3.5  5.2  1007.800000  64.0   
2025-03-28       0.1  -6.900000  48.000000  3.5  5.2  1005.500000  64.0   
2025-03-29       0.1  -5.700000   7.500000  3.5  5.2  1008.300000  64.0   
2025-03-30       0.1  -5.700000   7.500000  3.5  5.2  1008.300000  64.0   
2025-03-31       0.1  25.450000  86.250000  3.5  5.2  1010.150000  64.0   
2025-04-01       0.1  25.700000  56.000000  3.5  5.2  1011.000000  64.0   
2025-04-02       0.1  20.400000  40.300000  3.5  5.2  1010.300000  64.0   
2025-04-03       0.1  20.400000  64.000000  3.5  5.2  1012.600000  64.0   
2025-04-04       0.1  -5.800000   7.800000  3.5  5.2  1012.600000  64.0   
2025-04-05       0.1  -5.800000   7.700000  3.5  5.2  1011.500000  64.0   
2025-04-06       0.1  -5.800000   7.700000  3.5  5.2  1011.500000  64.0   
2025-04-07       0.1  -6.500000   8.000000  3.5  5.2  1009.350000  64.0   
2025-04-08       0.1  24.700000  55.000000  3.5  5.2  1010.700000  64.0   
2025-04-09       0.1  24.700000  64.000000  3.5  5.2  1009.800000  64.0   
2025-04-10       0.1  -6.

In [21]:
summary_statistics = resampled_aqi.describe().transpose().reset_index()

for metric in summary_statistics["level_1"].unique():
    print(f"Summary statistics of {metric}")
    display(summary_statistics[summary_statistics["level_1"] == metric])

Summary statistics of co


,City,level_1,count,mean,std,min,25%,50%,75%,max
0,Ayutthaya,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
12,Bangkok,co,52.0,0.1,4.203955e-17,0.1,0.1,0.1,0.1,0.1
44,Kanchanaburi,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
54,Lampang,co,52.0,0.1,4.203955e-17,0.1,0.1,0.1,0.1,0.1
77,Mueang Chiang Rai,co,52.0,0.1,4.203955e-17,0.1,0.1,0.1,0.1,0.1
114,Nakhon Sawan,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
126,Nan,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
146,Nong Khai,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
158,Nonthaburi,co,52.0,0.1,4.235278e-17,0.1,0.1,0.1,0.1,0.1
169,Pathum Thani,co,52.0,0.1,4.203955e-17,0.1,0.1,0.1,0.1,0.1


Summary statistics of dew


,City,level_1,count,mean,std,min,25%,50%,75%,max
1,Ayutthaya,dew,52.0,2.936859,12.268427,-7.4,-6.2500,-5.750,14.4000,25.70
33,Chonburi,dew,52.0,6.441346,12.238251,-8.0,-5.5500,8.875,16.4500,26.60
45,Kanchanaburi,dew,52.0,4.156410,11.690678,-8.2,-4.9250,-4.050,15.6000,24.90
55,Lampang,dew,52.0,3.100000,10.985623,-8.0,-4.7000,-3.800,16.2000,21.50
66,Lamphun,dew,52.0,12.619231,8.863741,-6.4,11.0250,16.500,19.0000,21.30
86,Mueang Khon Kaen,dew,52.0,7.299038,11.886854,-8.2,-4.9000,7.375,18.0000,25.10
94,Nakhon Pathom,dew,52.0,6.667308,12.922578,-8.8,-5.8000,8.750,20.2000,25.50
104,Nakhon Ratchasima,dew,52.0,2.745192,11.883603,-8.1,-5.0000,-4.350,8.7500,26.50
115,Nakhon Sawan,dew,52.0,5.355769,12.715197,-8.9,-5.6500,-3.800,20.5250,26.80
127,Nan,dew,52.0,1.707692,10.132589,-8.8,-5.4000,-4.900,8.9500,21.30


Summary statistics of h


...

,City,level_1,count,mean,std,min,25%,50%,75%,max
9,Ayutthaya,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
20,Bangkok,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
30,Chiang Mai,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
41,Chonburi,so2,52.0,1.100000,0.000000e+00,1.1,1.1000,1.10,1.100,1.10
63,Lampang,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
74,Lamphun,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
101,Nakhon Pathom,so2,52.0,9.000000,0.000000e+00,9.0,9.0000,9.00,9.000,9.00
111,Nakhon Ratchasima,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
123,Nakhon Sawan,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60
135,Nan,so2,52.0,0.600000,1.121055e-16,0.6,0.6000,0.60,0.600,0.60


Summary statistics of t


,City,level_1,count,mean,std,min,25%,50%,75%,max
10,Ayutthaya,t,52.0,31.698077,2.799947,24.5,30.100000,32.300,33.4000,36.6
21,Bangkok,t,52.0,32.896154,2.320177,26.0,31.562500,33.600,34.2000,36.7
31,Chiang Mai,t,52.0,31.696154,2.985994,24.8,29.950000,32.000,34.0000,36.4
42,Chonburi,t,52.0,31.527885,2.667661,23.7,29.475000,32.700,33.5000,36.2
52,Kanchanaburi,t,52.0,32.674359,3.482126,25.2,30.075000,34.000,35.0000,38.3
64,Lampang,t,52.0,30.602885,3.912654,21.9,28.175000,30.650,34.0250,38.2
75,Lamphun,t,52.0,31.069231,3.645870,22.0,28.875000,32.000,34.0000,38.2
84,Mueang Chiang Rai,t,52.0,30.268269,4.065747,17.7,28.637500,30.800,33.1500,35.6
92,Mueang Khon Kaen,t,52.0,30.323077,5.024913,19.0,26.000000,30.375,35.0500,37.0
102,Nakhon Pathom,t,52.0,32.015385,3.128718,23.3,30.800000,32.775,34.3500,36.0


Summary statistics of w


,City,level_1,count,mean,std,min,25%,50%,75%,max
11,Ayutthaya,w,52.0,2.415385,1.322080,0.2,1.500000,2.500000,3.0000,5.10
22,Bangkok,w,52.0,2.375000,1.120071,0.5,1.500000,2.000000,3.1500,4.60
32,Chiang Mai,w,52.0,1.490385,0.767479,0.5,1.000000,1.500000,2.0000,3.60
43,Chonburi,w,52.0,2.683654,1.231201,1.0,2.000000,2.500000,3.0000,6.10
53,Kanchanaburi,w,52.0,1.975962,0.911446,0.5,1.500000,2.000000,2.5000,5.10
65,Lampang,w,52.0,1.348077,0.929735,0.5,0.500000,1.000000,1.5000,5.00
76,Lamphun,w,52.0,1.816346,0.787134,0.5,1.425000,1.875000,2.1750,3.30
85,Mueang Chiang Rai,w,52.0,1.601923,0.851697,0.1,1.000000,1.625000,2.0000,3.60
93,Mueang Khon Kaen,w,52.0,1.793269,1.076397,0.5,1.000000,1.500000,2.1250,5.60
103,Nakhon Pathom,w,52.0,2.276923,1.033956,0.5,1.500000,2.500000,3.0000,4.60


## Hypothesis 1

In [22]:
# Get metadata
city_meta = df_no_missing[['City', 'City_region', 'City_type']].drop_duplicates().set_index("City")

results = []
feature = "pm25"

for region in city_meta["City_region"].unique():
    regional_cities = city_meta[city_meta["City_region"] == region]
    
    majors = regional_cities[regional_cities["City_type"] == "Major"].index
    peripherals = regional_cities[regional_cities["City_type"] == "Peripheral"].index

    for major_city, peripheral_city in product(majors, peripherals):
        try:
            series1 = resampled_aqi[(major_city, feature)].dropna()
            series2 = resampled_aqi[(peripheral_city, feature)].dropna()
        except KeyError:
            continue  # Skip if either city's data is missing

        # Align by common dates
        common_index = series1.index.intersection(series2.index)
        series1 = series1.loc[common_index]
        series2 = series2.loc[common_index]

        if len(series1) < 3 or len(series2) < 3:
            continue

        # Shapiro normality test
        shapiro1 = shapiro(series1)
        shapiro2 = shapiro(series2)

        if shapiro1.pvalue > 0.05 and shapiro2.pvalue > 0.05:
            levene_test = levene(series1, series2)
            ttest = ttest_ind(series1, series2, equal_var=(levene_test.pvalue > 0.05))

            results.append({
                "region": region,
                "major_city": major_city,
                "peripheral_city": peripheral_city,
                "test": "t-test",
                "p_value": ttest.pvalue,
                "shapiro1_p": shapiro1.pvalue,
                "shapiro2_p": shapiro2.pvalue,
                "levene_p": levene_test.pvalue,
            })
        else:
            u_test = mannwhitneyu(series1, series2, alternative="two-sided")
            results.append({
                "region": region,
                "major_city": major_city,
                "peripheral_city": peripheral_city,
                "test": "mannwhitneyu",
                "p_value": u_test.pvalue,
                "shapiro1_p": shapiro1.pvalue,
                "shapiro2_p": shapiro2.pvalue,
                "levene_p": None,
            })

# Wrap in DataFrame
comparison_results = pd.DataFrame(results)
significant_comparisons = comparison_results[comparison_results["p_value"] < 0.05]

In [23]:
comparison_results

,region,major_city,peripheral_city,test,p_value,shapiro1_p,shapiro2_p,levene_p
0,Central,Bangkok,Ayutthaya,mannwhitneyu,2.423012e-02,0.000836,0.168525,NaN
1,Central,Bangkok,Nakhon Pathom,mannwhitneyu,7.740254e-03,0.000836,0.009433,NaN
2,Central,Bangkok,Nakhon Sawan,mannwhitneyu,2.793078e-02,0.000836,0.002398,NaN
3,Central,Bangkok,Nonthaburi,mannwhitneyu,7.640886e-04,0.000836,0.000274,NaN
4,Central,Bangkok,Pathum Thani,mannwhitneyu,8.762573e-03,0.000836,0.004640,NaN
5,Central,Bangkok,Phitsanulok,mannwhitneyu,8.555075e-01,0.000836,0.015437,NaN
6,Central,Bangkok,Samut Prakan,mannwhitneyu,7.230096e-01,0.000836,0.001710,NaN
7,Central,Bangkok,Samut Sakhon,mannwhitneyu,2.046516e-05,0.000836,0.013579,NaN
8,Central,Bangkok,Saraburi,mannwhitneyu,7.775359e-05,0.000836,0.011795,NaN
9,Central,Bangkok,Suphan Buri,mannwhitneyu,2.926852e-06,0.000836,0.000053,NaN


In [24]:
significant_comparisons

,region,major_city,peripheral_city,test,p_value,shapiro1_p,shapiro2_p,levene_p
0,Central,Bangkok,Ayutthaya,mannwhitneyu,2.423012e-02,0.000836,0.168525,NaN
1,Central,Bangkok,Nakhon Pathom,mannwhitneyu,7.740254e-03,0.000836,0.009433,NaN
2,Central,Bangkok,Nakhon Sawan,mannwhitneyu,2.793078e-02,0.000836,0.002398,NaN
3,Central,Bangkok,Nonthaburi,mannwhitneyu,7.640886e-04,0.000836,0.000274,NaN
4,Central,Bangkok,Pathum Thani,mannwhitneyu,8.762573e-03,0.000836,0.004640,NaN
7,Central,Bangkok,Samut Sakhon,mannwhitneyu,2.046516e-05,0.000836,0.013579,NaN
8,Central,Bangkok,Saraburi,mannwhitneyu,7.775359e-05,0.000836,0.011795,NaN
9,Central,Bangkok,Suphan Buri,mannwhitneyu,2.926852e-06,0.000836,0.000053,NaN
10,North,Chiang Mai,Lampang,mannwhitneyu,7.084714e-05,0.002839,0.167802,NaN
11,North,Chiang Mai,Lamphun,mannwhitneyu,5.548520e-04,0.002839,0.013255,NaN


In [25]:
# export data for modeling
unstack_resampled = resampled_aqi.unstack().reset_index()
train_data = unstack_resampled.pivot_table(values=0, index=["City", "time"], columns=["level_1"]).reset_index()
train_data.to_csv("air_data_train.csv")